In [1]:
import os
import pandas as pd
from scipy.stats import kstest
import numpy as np
from scipy.stats import wilcoxon

root_path = "/Users/zeyuzhao/Reasearch/CCVL/ResultRanking/final_14dec2024"

score_types = ['dsc', 'nsd']
datasets = ['dapatlas_results', 'jhh_results', 'totalsegmentator_results']
organs = [
    "aorta", "gall_bladder", "kidney_left", "kidney_right",
    "liver", "pancreas", "postcava", "spleen", "stomach", "mean"
]

In [21]:
organization = {
 'nnU-Net_MedNeXt': 'DKFZ',
 'nnU-Net_STU-Net': 'Shanghai AI Lab',
 'nnU-Net_ResEncL': 'DKFZ',
 'nnU-Net_U-Net': 'N/A',
 'nnU-Net U-Net': 'N/A',
 '_Diff-UNet': 'HKUST',
 'MONAI_LHU-Net': 'UR',
 'nnU-Net_NexToU': 'HIT',
 'MONAI_SegVol': 'BAAI',
 'Vision_Language_U-Net_CLIP': 'CityU',
 'U-Net_CLIP': 'CityU',
 'MONAI_UCTransNet': 'CityU',
 'Vision_Language_Swin_UNETR_CLIP': 'CityU',
 'MONAI_UNEST': 'NVIDIA',
 'MONAI_Swin_UNETR': 'NVIDIA',
 'MONAI_UNETR': 'NVIDIA',
 '_SAM-Adapter': 'Duke',
  'MedNeXt': 'DKFZ',
 'STU-Net-B': 'Shanghai AI Lab',
 'STU-Net H': 'Shanghai AI Lab',
 'MedFormer': 'Rutgers',
 'nnU-Net ResEncL': 'DKFZ',
 'UniSeg': 'NPU',
 'Diff-UNet': 'HKUST',
 'LHU-Net': 'UR',
 'NexToU': 'HIT',
 'SegVol': 'BAAI',
 'U-Net & CLIP': 'CityU',
 'Swin UNETR & CLIP': 'CityU',
 'Swin_UNETR_CLIP': 'CityU',
 'Swin UNETR': 'NVIDIA',
 'UNesT': 'NVIDIA',
 'UNEST': 'NVIDIA',
 'SAM-Adapter': 'Duke',
 'UNETR': 'NVIDIA'
}


In [3]:
data = {}
for dataset in datasets:
    data[dataset] = {}
    models = [f for f in os.listdir(os.path.join(root_path, dataset)) if not f.startswith(".")]
    for model in models:
        data[dataset][model] = {}
        for score_type in score_types:
            path = os.path.join(root_path, dataset, model, f"{score_type}.csv")
            tmp = pd.read_csv(path)
            tmp["mean"] = tmp.iloc[:, 1:].mean(axis=1, skipna=False)
            data[dataset][model][score_type] = tmp


In [4]:
import pprint
pprint.pprint(data)

{'dapatlas_results': {'MONAI_LHU-Net': {'dsc':                          name     aorta  gall_bladder  kidney_left  \
0    AutoPET_099b3fd402_84728  0.731734      0.871979     0.909235   
1    AutoPET_0410759456_20805  0.928610      0.915884     0.952522   
2    AutoPET_07b7e9abfc_08717  0.873651      0.901934     0.922252   
3    AutoPET_07574bfa00_91235  0.802928      0.915135     0.972801   
4    AutoPET_04ab5c61c9_42241  0.876165      0.918996     0.971300   
..                        ...       ...           ...          ...   
528  AutoPET_fbd907a179_52084  0.932931      0.698327     0.976960   
529  AutoPET_faf4a6dbaa_49516  0.930705      0.894320     0.969890   
530  AutoPET_fde66dd53a_79975  0.932539           NaN     0.955720   
531  AutoPET_fe705ea1cc_47025  0.934992      0.896899     0.967510   
532  AutoPET_ff1451316e_22165  0.931810      0.902865     0.963427   

     kidney_right     liver  pancreas  postcava    spleen   stomach      mean  
0        0.944943  0.961551  0.7

In [5]:
import pandas as pd
from scipy.stats import wilcoxon
import os

# 假设 data 已加载
# 定义输出目录
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 遍历数据集和模型
likely_models = {}
for dataset_name, models in data.items():
    likely_models[dataset_name] = {}
    for metric in ['dsc', 'nsd']:  # 按指标分开计算
        likely_models[dataset_name][metric] = {}
        for organ in organs:
            likely_models[dataset_name][metric][organ] = {}
            for model1, metrics1 in models.items():
                model1_list = []
                if metric not in metrics1:
                    continue
                df1 = metrics1[metric] 

                for model2, metrics2 in models.items():
                    if model1 == model2:  # 跳过对自身模型的比较
                        continue
                    
                    df2 = metrics2[metric]

                    # 获取两组数据
                    data1 = df1[organ]
                    data2 = df2[organ]

                    # 确保配对数据长度一致
                    paired_data = pd.DataFrame({'data1': data1, 'data2': data2}).dropna()
                    if paired_data.empty:
                        continue

                    # 使用 Wilcoxon 检验
                    try:
                        stat, p_value = wilcoxon(paired_data['data1'], paired_data['data2'])
                        if p_value > 0.05:  # 无显著差异
                            model1_list.append(model2)
                    except ValueError:
                        continue
                likely_models[dataset_name][metric][organ][model1] = model1_list
print(likely_models)


{'dapatlas_results': {'dsc': {'aorta': {'MONAI_UNEST': [], 'MONAI_LHU-Net': ['nnU-Net_ResEncL', 'MONAI_UCTransNet', 'nnU-Net_NexToU', '_Diff-UNet', 'nnU-Net_U-Net'], 'Vision_Language_U-Net_CLIP': [], 'nnU-Net_STU-Net': ['nnU-Net_MedNeXt'], 'nnU-Net_ResEncL': ['MONAI_LHU-Net', 'MONAI_UCTransNet', '_Diff-UNet', 'nnU-Net_U-Net'], 'nnU-Net_MedNeXt': ['nnU-Net_STU-Net'], 'MONAI_UNETR': [], 'Vision_Language_Swin_UNETR_CLIP': [], 'MONAI_SegVol': [], 'MONAI_UCTransNet': ['MONAI_LHU-Net', 'nnU-Net_ResEncL', 'nnU-Net_U-Net'], 'nnU-Net_NexToU': ['MONAI_LHU-Net', '_Diff-UNet'], '_SAM-Adapter': [], '_Diff-UNet': ['MONAI_LHU-Net', 'nnU-Net_ResEncL', 'nnU-Net_NexToU'], 'MONAI_Swin_UNETR': [], 'nnU-Net_U-Net': ['MONAI_LHU-Net', 'nnU-Net_ResEncL', 'MONAI_UCTransNet']}, 'gall_bladder': {'MONAI_UNEST': ['MONAI_Swin_UNETR'], 'MONAI_LHU-Net': ['nnU-Net_STU-Net', 'nnU-Net_ResEncL', 'nnU-Net_MedNeXt', 'MONAI_UCTransNet', '_Diff-UNet'], 'Vision_Language_U-Net_CLIP': ['Vision_Language_Swin_UNETR_CLIP', 'MONAI_

In [6]:
import pprint
pprint.pprint(likely_models)

{'dapatlas_results': {'dsc': {'aorta': {'MONAI_LHU-Net': ['nnU-Net_ResEncL',
                                                          'MONAI_UCTransNet',
                                                          'nnU-Net_NexToU',
                                                          '_Diff-UNet',
                                                          'nnU-Net_U-Net'],
                                        'MONAI_SegVol': [],
                                        'MONAI_Swin_UNETR': [],
                                        'MONAI_UCTransNet': ['MONAI_LHU-Net',
                                                             'nnU-Net_ResEncL',
                                                             'nnU-Net_U-Net'],
                                        'MONAI_UNEST': [],
                                        'MONAI_UNETR': [],
                                        'Vision_Language_Swin_UNETR_CLIP': [],
                                        'Vision_Language_U-Net_

In [25]:
root_path = "/Users/zeyuzhao/Reasearch/CCVL/static/front"
for dataset_name, models in data.items():
    for metric in ['dsc', 'nsd']:  # 按指标分开计算
        for organ in organs:
            results = []
            for model_name, metrics in models.items():
                df = metrics[metric]
                scores = df[organ]
                median = np.nanmedian(scores)
                lower_bound = np.nanpercentile(scores, 2.5)
                upper_bound = np.nanpercentile(scores, 97.5)    
                models_for_model = likely_models[dataset_name][metric][organ][model_name]
                results.append({
                        'Model_name': model_name,
                        'Median': median,
                        '95% CI': f"{lower_bound:.2f} - {upper_bound:.2f}",
                        'Likely_models': models_for_model,
                        'organization': organization.get(model_name, "Collecting Data")
                    })
            # 将结果整理成 DataFrame
            results_df = pd.DataFrame(results)

            # 保存结果到文件
            cur_path = os.path.join(root_path, dataset_name, metric)
            os.makedirs(cur_path, exist_ok=True)
            output_file = os.path.join(cur_path, f"{organ}_table.csv")
            results_df.to_csv(output_file, index=False)
            print(f"无显著差异的模型结果已保存至: {output_file}")
            # print(results_df)
                